In [1]:
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex
import glob
import os
import fitz

c:\Users\josee\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Ruta al directorio que contiene los archivos PDF
pdf_dir = "C:/Users/josee/OneDrive/Documents/8vo Semestre/Reto/RetoCEMEX/PDF"

# Cargar el modelo SentenceTransformer
model_path = "bert-large-uncased"
model = SentenceTransformer(model_path)

# Función para extraer texto de un archivo PDF
def extraer_texto_de_pdf(pdf_path, max_length=512):
    texto = ""
    try:
        with fitz.open(pdf_path) as archivo_pdf:
            for num_pagina in range(len(archivo_pdf)):
                pagina = archivo_pdf[num_pagina]
                texto_pagina = pagina.get_text()
                # Truncar el texto de la página si es necesario
                if len(texto) + len(texto_pagina) > max_length:
                    texto_pagina = texto_pagina[:max_length - len(texto)]
                texto += texto_pagina
                
        embeddings = model.encode([texto])[0]
                
    except Exception as e:
        print(f"Error al leer el archivo PDF {pdf_path}: {e}")
    return texto


# Función para buscar en los archivos PDF
def buscar_en_archivos_pdf(consulta):
    archivos = glob.glob(os.path.join(pdf_dir, "*.pdf"))

    # Indexar archivos PDF
    annoy_index = AnnoyIndex(1024, metric='angular')
    for idx, archivo in enumerate(archivos):
        texto = extraer_texto_de_pdf(archivo)
        incrustacion = model.encode([texto])[0]
        annoy_index.add_item(idx, incrustacion)
    annoy_index.build(10)

    # Codificar la consulta y encontrar archivos PDF similares
    incrustacion_consulta = model.encode([consulta])[0]
    indices_mas_cercanos, distancias_mas_cercanas = annoy_index.get_nns_by_vector(incrustacion_consulta, 5, include_distances=True)

    # Mostrar o procesar los archivos PDF más cercanos
    resultados = []
    for idx, distancia in zip(indices_mas_cercanos, distancias_mas_cercanas):
        resultados.append((archivos[idx], distancia))
    return resultados



No sentence-transformers model found with name bert-large-uncased. Creating a new one with MEAN pooling.
c:\Users\josee\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


: 

In [ ]:
consulta = input("Palabra Clave sobre lo que busca: ")

In [9]:
# Consulta de búsqueda
#consulta = "EJEMPLO DE FACTURA SIMPLE"

# Realizar la búsqueda
resultados = buscar_en_archivos_pdf(consulta)

# Mostrar resultados
for resultado in resultados:
    print(resultado)

('C:/Users/josee/OneDrive/Documents/8vo Semestre/Reto/RetoCEMEX/PDF\\Ejemplo3.pdf', 0.7202630043029785)
('C:/Users/josee/OneDrive/Documents/8vo Semestre/Reto/RetoCEMEX/PDF\\Ejemplo2.pdf', 0.9634143710136414)
('C:/Users/josee/OneDrive/Documents/8vo Semestre/Reto/RetoCEMEX/PDF\\Ejemplo1.pdf', 0.9785165786743164)
